<a href="https://colab.research.google.com/github/DamnScallion/Agents-Demo/blob/main/Agent_Demo_2_Open_AI_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai-agents gradio -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.4/100.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 106.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "REPLACE THIS STRING WITH YOUR OPEN API KEY"

# Apply nest_asyncio to integrate with Jupyter's event loop
import nest_asyncio
nest_asyncio.apply()

## The simplest Agent

In [ ]:
from agents import Agent, Runner
import nest_asyncio

agent = Agent(name="Assistant", instructions="You are a helpful assistant")

result = Runner.run_sync(agent, "Write a haiku about generative AI.")
print(result.final_output)

# Code within the code,
# Functions calling themselves,
# Infinite loop's dance.

Machines dream in code,  
Creating worlds from patterns—  
Silent sparks ignite.


## Homework multi-agent with guardrail

In [ ]:
from agents import Agent, InputGuardrail,GuardrailFunctionOutput, Runner
from pydantic import BaseModel

class HomeworkOutput(BaseModel):
    is_homework: bool
    reasoning: str

math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
)

history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
)

guardrail_agent = Agent(
    name="Guardrail check",
    instructions="Check if the user is asking about homework.",
    output_type=HomeworkOutput,
)

async def homework_guardrail(ctx, agent, input_data):
    result = await Runner.run(guardrail_agent, input_data, context=ctx.context)
    final_output = result.final_output_as(HomeworkOutput)

    return GuardrailFunctionOutput(
        output_info=final_output,
        tripwire_triggered=not final_output.is_homework,
    )

triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question",
    handoffs=[history_tutor_agent, math_tutor_agent],
    input_guardrails=[
        InputGuardrail(guardrail_function=homework_guardrail),
    ],
)


In [ ]:
result = await Runner.run(triage_agent, "who was the first president of china in 1949?")
print(result)
# print(result.final_output) # just output the answer

RunResult:
- Last agent: Agent(name="History Tutor", ...)
- Final output (str):
    In 1949, the People's Republic of China was founded, and Mao Zedong became its first leader. Officially, he held the title of Chairman of the Central People's Government. Later, he was also known as the Chairman of the Communist Party of China, which was the most powerful position in the country. The formal title of "President" wasn’t initially used until later under the current constitutional framework.
- 3 new item(s)
- 2 raw response(s)
- 1 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


In [ ]:
# Is this a homework question?
result = await Runner.run(triage_agent, "what is life")
print(result.final_output)

InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire

### Simple web search example

In [ ]:
from agents import Agent, Runner, WebSearchTool, trace

search_agent = Agent(
      name="Web searcher",
      instructions="You are a helpful agent.",
      tools=[WebSearchTool(user_location={"type": "approximate", "city": "Sydney"})],
  )


In [ ]:
result = await Runner.run(
    search_agent,
    "search the web for 'local technology news' today and give me 1 interesting update in a sentence.",
)

print(result.final_output)
# The New York Giants are reportedly pursuing quarterback Aaron Rodgers after his ...


As of May 4, 2025, Sydney is hosting several significant technology events, including the Techstars Startup Weekend Sydney 2025 from April 11-13, aimed at fostering new business ideas, and the Future of Government: Security conference on April 2, focusing on enhancing cyber resilience in government systems.  


## Make a user interface for the agent

In [ ]:
from agents import Agent, Runner
import gradio as gr

# Function to run the agent and get the response
async def get_agent_response(user_input):
    result = await Runner.run(search_agent, user_input)
    return result.final_output

# Create a Gradio interface
iface = gr.Interface(
    fn=get_agent_response,
    inputs=gr.Textbox(lines=2, placeholder="Enter your message here..."),
    outputs="text",
    title="Simple Chatbot Demo",
    description="Ask your questions to the helpful assistant.",
)

# Launch the interface
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://83ce395d192424533b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
